In [ ]:
import numpy as np
A = np.array([[0.9512, 0],
              [0.0476, 0.9512]], ndmin = 2)
B = np.array([[1], [2]], ndmin = 2)
C = np.array([0, 1], ndmin = 2)

mean_w = np.array([[0], [0]], ndmin = 2)
variance_w = 1e-3 * np.array([[9.506, 0.234],
                              [0.234, 9.512]], ndmin = 2)
mean_v = 0
variance_v = 0.0125

h = 0.1 # sampling period in second

mean_x0 = np.array([[5], [5]], ndmin = 2)
variance_x0 = np.array([[1, 0],
                        [0, 1]], ndmin = 2)

print(A)
print(B)
print(C)
print(mean_w)
print(variance_w)